# A press request for Mapping Police Violence
We occasionally get press requests which require quick analyses of our data. Here is an example of a real request we got: “I am working on a story about police traffic stops. We were wondering if Mapping Police Violence happens to have data on the number of civilian deaths that occur during traffic stops each year.”

Mapping Police Violence (MPV) is a public dataset that CZ runs that tracks lives lost during police encounters in the US and compiles information around the circumstances of these incidents.

Trigger warning: this data contains demographic information about the civilians killed and information about the manner in which they were killed. The circumstances field contains brief descriptions of the incidents, some of which are graphic in detail.

# Part 0 - Load Data

In [339]:
import pandas as pd
import numpy as np
police_killings = pd.read_csv("police_killings.csv", low_memory=False)
print(police_killings.shape)
police_killings.head()

(10345, 68)


,name,age,gender,race,date,street_address,city,state,zip,county,...,prosecutor_gender,prosecutor_special,prosecutor_url,prosecutor_in_court,possible_suicide,alert_ids,prosecutor_party,prosecutor_term,independent_investigation,officer_race_inferred
0,Joseph Fuller,24.0,Male,White,2015-07-21,1300 NE 50th Ct,Oakland Park,FL,33334.0,Broward,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Shane K. Jones,38.0,Male,Black,2020-11-16,161 SW 19th Ct.,Dania Beach,FL,330004.0,Broward,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,John Ethan Carpentier,26.0,Male,White,2016-09-27,20838 N 19th Ave,Phoenix,AZ,85027.0,Maricopa,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Mickee McArthur,28.0,Male,Black,2020-11-30,N. 9th Ave. and Langley Ave.,Ferry Pass,FL,32504.0,Escambia,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Mark Bess,33.0,Male,White,2016-04-30,4500 S 3200 W,Spanish Fork,UT,84660.0,Utah,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [340]:
police_killings.loc[0]

name                         Joseph Fuller
age                                   24.0
gender                                Male
race                                 White
date                            2015-07-21
                                 ...      
alert_ids                              NaN
prosecutor_party                       NaN
prosecutor_term                        NaN
independent_investigation              NaN
officer_race_inferred                  NaN
Name: 0, Length: 68, dtype: object

## Part 1

The police_killings.csv file provided to you contains a snapshot of this data. Each row corresponds to one victim from one incident. This file includes both the finalized data that we’ve made public, and internal-only, incomplete data that is in the process of being populated before we publish it. See codebook.pdf for a comprehensive list of each field and its explanation.


#### How many traffic stop-involved police killings (TSPKs) are there in total?

In [341]:
# TSPKs in total
options = ['Traffic Stop', 'Traffic Stop/Other Non-Violent Offense']
tspks = pd.DataFrame(police_killings[police_killings['encounter_type'].isin(options)])
print('TSPKs =', len(tspks))
# response time: 10 min

TSPKs = 644


#### What proportion of all police killings do TSPKs comprise?

In [342]:
proportion_tspks = len(tspks) / len(police_killings)
print('Proportion TSPKs/PoliceKillings =', "{0:.2%}".format(proportion_tspks))
# response time: 5 min

Proportion TSPKs/PoliceKillings = 6.23%


#### Briefly describe how you arrived at these numbers and explain any decisions you made in the process.

To determine the numbers I first filtered the 'encounter_type' column for 'Traffic Stop' and 'Traffic Stop/Other Non-Violent Offense'. The length of this dataset would correspond to the TSPKs. To find the proportion I have divided the TSPKs by the total length of the dataset 'police_killings'

Response time: 5 min

## Part 2

#### What is the total number of incidents each year?

In [343]:
tspks['date'] = pd.to_datetime(tspks['date'])
incidents_by_year = tspks.groupby([tspks['date'].dt.year])['name'].count().reset_index()
incidents_by_year = incidents_by_year.rename(columns={'date': 'year', 'name': '# incidents'})
incidents_by_year

# response time: 10 min
# resource:  https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.groupby.html

,year,# incidents
0,2017,101
1,2018,123
2,2019,130
3,2020,131
4,2021,131
5,2022,28


#### How do TSPKs break down by race?

In [344]:
tspks_by_race = tspks.groupby(tspks['race'])['name'].count().reset_index()
tspks_by_race = tspks_by_race.rename(columns={ 'name': '# incidents'})
tspks_by_race

# response time: 10 min

,race,# incidents
0,Asian,10
1,Black,183
2,Hispanic,101
3,Native American,13
4,Pacific Islander,6
5,Unknown race,71
6,White,253


#### What are the top 3 agencies responsible for the most TSPKs, and how many TSPKs are each responsible for?

In [345]:
top_agencies = tspks.groupby(tspks['agency_responsible'])['name'].count().reset_index()
top_agencies = top_agencies.rename(columns={ 'name': '# incidents'}).sort_values(by='# incidents', ascending=False)
top_agencies.iloc[:3]

# response time: 10 min
# resource: https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.sort_values.html

,agency_responsible,# incidents
106,California Highway Patrol,9
257,Los Angeles County Sheriff's Department,8
65,Arizona Department of Public Safety,7


## Part 3
To prepare to launch a campaign on TSPKs, the front-end developers need 2 tables:
* For every state, the absolute number of TSPKs, broken down by race as well as the total for all races.
* The same table, but with TSPKs per capita instead of absolute numbers.

a. Create the two tables as tspk_abs.csv and tspk_percap.csv.
b. Which state has the highest per capita rate of TSPKs overall and what is it?
c. Which state has the lowest per capita rate, and what is it?
d. How do these numbers compare to the national per capita rate?

Use race_eth_by_state_2020_census.csv (provided) to compute TSPKs per capita.

In [346]:
tspk_abs = tspks.groupby(['state', 'race'])['name'].count().reset_index()
tspk_abs = tspk_abs.rename(columns={ 'name': '# incidents'})

tspk_abs = pd.pivot_table(tspk_abs, values='# incidents', index=['state'], columns='race', aggfunc=np.sum, fill_value=0)
tspk_abs.loc["Total"] = tspk_abs.sum()
tspk_abs['Total'] = tspk_abs.sum(axis=1)
tspk_abs

# response time: 20 min
# resources: https://pandas.pydata.org/docs/reference/api/pandas.pivot_table.html
#            https://stackoverflow.com/questions/21752399/pandas-dataframe-total-row
#            https://stackoverflow.com/questions/20804673/appending-column-totals-to-a-pandas-dataframe

race,Asian,Black,Hispanic,Native American,Pacific Islander,Unknown race,White,Total
state,,,,,,,,
AK,0,0,0,2,1,0,1,4
AL,0,6,1,0,0,0,8,15
AR,1,6,0,0,0,3,5,15
AZ,0,4,10,1,0,3,5,23
CA,3,10,33,1,1,12,20,80
CO,0,2,4,1,0,3,16,26
CT,0,1,1,0,0,0,0,2
DC,0,1,0,0,0,0,0,1
DE,0,1,0,0,0,0,2,3


In [347]:
race_eth_by_state_2020_census = pd.read_csv('race_eth_by_state_2020_census.csv')
race_eth_by_state_2020_census = race_eth_by_state_2020_census.drop(0).set_index('GEO_ID').apply(pd.to_numeric, errors='ignore')
race_eth_by_state_2020_census = race_eth_by_state_2020_census.rename(columns={'ASIAN':'Asian', 'BLACK':'Black', 'HISPANIC':'Hispanic', 'WHITE':'White', 'NATIVE_AMERICAN':'Native American', 'PACIFIC_ISLANDER':'Pacific Islander', 'OTHER': 'Unknown race', 'TOTAL':'Total'})
race_eth_by_state_2020_census.loc["Total"] = race_eth_by_state_2020_census.sum()
race_eth_by_state_2020_census

,NAME,Total,Hispanic,White,Black,Native American,Asian,Pacific Islander,Unknown race
GEO_ID,,,,,,,,,
0400000US01,Alabama,5024279,264047,3171351,1288159,23119,75918,2612,199073
0400000US02,Alaska,733391,49824,421758,20731,108838,43449,12455,76336
0400000US04,Arizona,7151502,2192253,3816547,317161,263930,248837,14323,298451
0400000US05,Arkansas,3011524,256847,2063550,449884,20549,51210,14280,155204
0400000US06,California,39538223,15579652,13714587,2119286,156085,5978795,138167,1851651
0400000US08,Colorado,5773714,1263390,3760663,221310,33768,195220,9005,290358
0400000US09,Connecticut,3605944,623293,2279232,360937,6404,170459,974,164645
0400000US10,Delaware,989948,104290,579851,212960,2521,42398,304,47624
0400000US11,District of Columbia,689545,77652,261771,282066,1277,33192,349,33238


In [348]:
state_codes = {
    'WA': '53', 'DE': '10', 'DC': '11', 'WI': '55', 'WV': '54', 'HI': '15',
    'FL': '12', 'WY': '56', 'PR': '72', 'NJ': '34', 'NM': '35', 'TX': '48',
    'LA': '22', 'NC': '37', 'ND': '38', 'NE': '31', 'TN': '47', 'NY': '36',
    'PA': '42', 'AK': '02', 'NV': '32', 'NH': '33', 'VA': '51', 'CO': '08',
    'CA': '06', 'AL': '01', 'AR': '05', 'VT': '50', 'IL': '17', 'GA': '13',
    'IN': '18', 'IA': '19', 'MA': '25', 'AZ': '04', 'ID': '16', 'CT': '09',
    'ME': '23', 'MD': '24', 'OK': '40', 'OH': '39', 'UT': '49', 'MO': '29',
    'MN': '27', 'MI': '26', 'RI': '44', 'KS': '20', 'MT': '30', 'MS': '28',
    'SC': '45', 'KY': '21', 'OR': '41', 'SD': '46',
}

def get_population(state, race):
    for item in race_eth_by_state_2020_census.index:
        if item == 'Total':
            return int(race_eth_by_state_2020_census.loc[item][race])
        elif str(item)[-2:] == state_codes.get(state):
            return int(race_eth_by_state_2020_census.loc[item][race])
    return 0

In [349]:
tspk_percap = tspk_abs.copy().astype('float')
for col in tspk_percap.columns:
    for row in tspk_percap.index:
        tspk_percap._set_value(row, col, tspk_percap.loc[[row]][col][0] / get_population(row, col) * 100000)
        # Taking into account that the values are very small, I have multiplied them by 100,000 to make them easier to read.

tspk_percap

# response time: 35 min
# resources: https://gist.github.com/wavded/1250983/bf7c1c08f7b1596ca10822baeb8049d7350b0a4b
#            https://www.indeed.com/career-advice/career-development/how-to-calculate-per-capita

race,Asian,Black,Hispanic,Native American,Pacific Islander,Unknown race,White,Total
state,,,,,,,,
AK,0.000000,0.000000,0.000000,1.837593,8.028904,0.000000,0.237103,0.545412
AL,0.000000,0.465781,0.378720,0.000000,0.000000,0.000000,0.252258,0.298550
AR,1.952744,1.333677,0.000000,0.000000,0.000000,1.932940,0.242301,0.498087
AZ,0.000000,1.261189,0.456152,0.378888,0.000000,1.005190,0.131008,0.321611
CA,0.050177,0.471857,0.211815,0.640677,0.723762,0.648070,0.145830,0.202336
CO,0.000000,0.903710,0.316608,2.961384,0.000000,1.033207,0.425457,0.450317
CT,0.000000,0.277057,0.160438,0.000000,0.000000,0.000000,0.000000,0.055464
DC,0.000000,0.354527,0.000000,0.000000,0.000000,0.000000,0.000000,0.145023
DE,0.000000,0.469572,0.000000,0.000000,0.000000,0.000000,0.344916,0.303046


### a. Create the two tables as tspk_abs.csv and tspk_percap.csv.


In [350]:
tspk_abs.to_csv('tspk_abs.csv')
tspk_percap.to_csv("tspk_percap.csv")

### b. Which state has the highest per capita rate of TSPKs overall and what is it?

In [351]:
highest = tspk_percap.sort_values(by='Total', ascending=False).iloc[:1]
highest['Total']

state
OK    0.555646
Name: Total, dtype: float64

### c. Which state has the lowest per capita rate, and what is it?

In [352]:
lowest = tspk_percap.sort_values(by='Total').iloc[:1]
lowest['Total']

state
MA    0.02845
Name: Total, dtype: float64

#### d. How do these numbers compare to the national per capita rate?

In [353]:
print('The highest per capita state is ',"{0:.2%}".format(highest['Total'][0]/ tspk_percap['Total'][-1]-1),'% more compare to the national rate of TSPKs')

The highest per capita state is  191.98% % more compare to the national rate of TSPKs


In [354]:
print('The lowest per capita state is ',"{0:.2%}".format(lowest['Total'][0]/ tspk_percap['Total'][-1]-1),'% less compare to the national rate of TSPKs')

The lowest per capita state is  -85.05% % less compare to the national rate of TSPKs


## Part 4
 Do you have any concerns about the data used to generate the results in Parts 1-3? If so, what are they?

To find the proportion I have divided the TSPKs by the total length of the dataset 'police_killings', however I have found a high value of null values in this column. If I determined the proportion based only on the non-null records in the ''encounter_type' column the value would change to:

Response time: 10 min

In [355]:
print("Nulls in 'encounter_type' column: ", len(police_killings[police_killings['encounter_type'].isnull()]))

proportion_tspks = len(tspks) / (len(police_killings) - len(police_killings[police_killings['encounter_type'].isnull()]))
print('Proportion TSPKs/PoliceKillings =', "{0:.2%}".format(proportion_tspks))

Nulls in 'encounter_type' column:  4524
Proportion TSPKs/PoliceKillings = 11.06%


# Part 5
We just noticed some agency_responsible fields were not input correctly. Some entries have trailing spaces, and some fields have multiple agencies entered, separated by a comma. This may have impacted the answers we got in Part 2.

Write a script that re-computes the top 3 agencies and their TSPK counts, correcting for these data entry issues, and attach it with the rest of your code and answers.


In [356]:
agencies = []
for item in tspks['agency_responsible']:
    agencies += str(item).strip().replace('"',"").split(",")

top_agencies =  pd.Series(agencies)
top_agencies = top_agencies.groupby(agencies).count()
top_agencies = top_agencies.rename('# incidents')
top_agencies.sort_values(ascending=False)[:3]

# response time: 15 min

Georgia State Patrol                       12
California Highway Patrol                  10
Los Angeles County Sheriff's Department     8
Name: # incidents, dtype: int64

#### Does making this correction change the answer in Part 2?

Yes, Now Georgia State Patrol is listed first. Followed by California Highway Patrol and Arizona Department of Public Safety.